# Validation

## Initialize

### Import packages

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, brier_score_loss
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample

### Load data

In [19]:
df = pd.read_excel(r"Data")

### Set standards

In [4]:
# Define color mapping for plotting
color_mapping = {
    "Bollen": "black",
    "Mod_Bauer": "blue",
    "Mizumoto": "red",
    "NESMS": "purple",
    "Orig_Bauer": "gold",
    "OSRI": "lime",
    "PathFx": "magenta",
    "Rev_Kat_individual": "orange",
    "Rev_Kat_grouped": "teal",
    "Rev_Toku": "cyan",
    "Tomita": "olive",
    "van_der_Linden": "brown",
    "SORG": "pink"}

# List of algorithms and time periods
algorithms = ["Bollen", "Mod_Bauer", "Mizumoto", "NESMS", "Orig_Bauer", 
              "OSRI", "PathFx", "Rev_Kat_individual", "Rev_Kat_grouped", "Rev_Toku",
              "Tomita", "van_der_Linden", "SORG"]
periods = ["3_months", "6_months", "12_months"]

In [4]:
# Making sure probabilities are between 0 and 1
for algorithm in algorithms:
    for period in periods:
        df[f"{algorithm}_{period}"] = (df[f"{algorithm}_{period}"])/100

## Define methods

In [ ]:
def calculate_metric_statistics(scores):
    """
    Calculate and return the mean and 95% confidence interval for a list of scores.
    
    Parameters:
    scores (array-like): A list or array of numerical scores for which the statistics are to be calculated.

    Returns:
    tuple: A tuple containing:
        - mean_score (float): The mean (average) of the scores, rounded to two decimal places.
        - lower_ci (float): The lower bound of the 95% confidence interval (2.5th percentile), rounded to two decimal places.
        - upper_ci (float): The upper bound of the 95% confidence interval (97.5th percentile), rounded to two decimal places.
    """

    LOWER_PERCENTILE = 2.5
    UPPER_PERCENTILE = 97.5

    sorted_scores = np.sort(scores)
    mean_score = round(np.mean(sorted_scores), 2)
    lower_ci = round(np.percentile(sorted_scores, LOWER_PERCENTILE), 2)
    upper_ci = round(np.percentile(sorted_scores, UPPER_PERCENTILE), 2)
    return mean_score, lower_ci, upper_ci

def logit(p):
    """
    Compute the logit (log-odds) of the given probabilities.
    
    This function calculates the logit, or the natural logarithm of the odds, for each probability in the input.
    Probabilities are clipped to avoid issues with extreme values (0% or 100%).

    Parameters:
    p (array-like): An array of probabilities (values between 0 and 1).

    Returns:
    array: An array of logit values corresponding to the input probabilities.
    """

    clipped_p = np.clip(p, 1e-15, 1 - 1e-15)
    return np.log(clipped_p / (1 - clipped_p)).values

def calibration_intercept_slope(y_true, y_pred_proba):
    """
    Calculate the calibration intercept and slope for predicted probabilities.

    This function fits a logistic regression model to the predicted probabilities to determine the calibration
    intercept and slope. The intercept and slope provide insight into the calibration of the predicted probabilities
    compared to the true outcomes.

    Parameters:
    y_true (array-like): An array of true binary outcomes (0 or 1).
    y_pred_proba (array-like): An array of predicted probabilities for the positive class.

    Returns:
    tuple: A tuple containing:
        - intercept (float): The intercept of the logistic regression model.
        - slope (float): The slope of the logistic regression model.
    """
    # Fit logistic regression to predicted probabilities
    log_odds = logit(y_pred_proba)
    
    lr = LogisticRegression(penalty=None)
    lr.fit(log_odds.reshape(-1, 1), y_true)
    
    # Intercept and slope
    intercept = lr.intercept_[0]
    slope = lr.coef_[0][0]
    
    return intercept, slope

def calc_measures(y_actual, y_pred_prob, algorithm):
    """
    Calculate performance measures for a given set of predicted probabilities using bootstrapping.

    This function performs bootstrapping to estimate the performance metrics including AUC, calibration intercept,
    calibration slope, and Brier score. It calculates the mean and confidence intervals for these metrics over 
    multiple bootstrap samples.

    Parameters:
    y_actual (array-like): An array of true binary outcomes (0 or 1).
    y_pred_prob (array-like): An array of predicted probabilities for the positive class.
    algorithm (str): The name of the algorithm being evaluated.

    Returns:
    pd.DataFrame: A DataFrame containing the mean and confidence intervals for AUC, calibration intercept,
                  calibration slope, and Brier score, labeled with the name of the algorithm.
    """
    
    # Bootstraps
    n_bootstraps = 2000

    # Set empty lists
    auc_scores = []
    calibration_intercepts = []
    calibration_slopes = []
    brier_scores = []
    
    # Bootstrap loop
    for i in range(n_bootstraps):
        # Bootstrap by sampling with replacement on the prediction indices, stratified on y_actual
        y_actual_boot, y_pred_prob_boot = resample(y_actual, y_pred_prob, 
                                                     replace=True, 
                                                     n_samples=len(y_actual), 
                                                     stratify=y_actual,
                                                     random_state=i)
                
        # Discrimination
        auc_scores.append(roc_auc_score(y_actual_boot, y_pred_prob_boot))
        
        # Calibration
        intercept, slope = calibration_intercept_slope(y_actual_boot, y_pred_prob_boot)
        calibration_intercepts.append(intercept)
        calibration_slopes.append(slope)
        
        # Brier score
        brier_scores.append(brier_score_loss(y_actual_boot, y_pred_prob_boot))

    # Calculate mean and confidence intervals (AUC, Calibration Intercept & Slope)
    auc_stat = calculate_metric_statistics(auc_scores)
    intercept_stat = calculate_metric_statistics(calibration_intercepts)
    slope_stat = calculate_metric_statistics(calibration_slopes)
    brier_stat = calculate_metric_statistics(brier_scores)

    # Compile results
    results_df = pd.DataFrame({
        'Algorithm': f"{algorithm}",
        'auc_score': [f"{auc_stat[0]} ({auc_stat[1]} - {auc_stat[2]})"],
        'calibration_intercept': [f"{intercept_stat[0]} ({intercept_stat[1]} - {intercept_stat[2]})"],
        'calibration_slope': [f"{slope_stat[0]} ({slope_stat[1]} - {slope_stat[2]})"],
        'brier_score': [f"{brier_stat[0]} ({brier_stat[1]} - {brier_stat[2]})"]})
    
    return results_df

## Validation of models for 3, 6, and 12 months survival

### 3 months

In [ ]:
results_3_months = pd.DataFrame()
for algorithm in algorithms:
    col_name = algorithm + "_3_months"
    result = calc_measures(df["3_months"], df[col_name], algorithm)
    results_3_months = pd.concat([results_3_months, result], ignore_index=True)
results_3_months

### 6 months

In [ ]:
results_6_months = pd.DataFrame()
for algorithm in algorithms:
    if algorithm == "SORG":
        continue  # Skip SORG algorithm (no SORG prediction at 6_months)

    col_name = algorithm + "_6_months"
    result = calc_measures(df["6_months"], df[col_name], algorithm)
    results_6_months = pd.concat([results_6_months, result], ignore_index=True)
results_6_months

### 12 months

In [ ]:
results_12_months = pd.DataFrame()
for algorithm in algorithms:
    col_name = algorithm + "_12_months"
    result = calc_measures(df["12_months"], df[col_name], algorithm)
    results_12_months = pd.concat([results_12_months, result], ignore_index=True)
results_12_months

### Save results

In [13]:
results_3_months.to_excel("Data_3_months")
results_6_months.to_excel("Data_6_months")
results_12_months.to_excel("Data_12_months")

In [10]:
results = {
    'results_3_months': pd.read_excel("Data_3_months"),
    'results_6_months': pd.read_excel("Data_6_months"),
    'results_12_months': pd.read_excel("Data_12_months")
}

## Complete case analysis

In [7]:
# Define algorithms to perform complete case analysis on
algorithms = ["Mizumoto", "NESMS", "PathFx", "Rev_Kat_grouped", "Rev_Kat_individual", "SORG"]
periods = ["3_months", "6_months", "12_months"]

In [9]:
for algorithm in algorithms:
    # Load the dataset
    df = pd.read_excel(rf"Data_algorithm_completecases") # Make sure you select complete cases seperate for each algorithm

    # Initialize an empty dictionary to store DataFrames for each timepoint
    results_dict = {}

    for period in periods:
        if algorithm == "SORG" and period == "6_months":
            continue # Skip SORG algorithm (no SORG prediction at 6_months)
            
        col_name = f"{algorithm}_{period}"
        df[col_name] = df[col_name]/100
        result = calc_measures(df[period], df[col_name], algorithm)
        results_dict[period] = result

    # Create a new Excel writer
    writer = pd.ExcelWriter(rf'Results\Complete case analysis\{algorithm}.xlsx', engine='xlsxwriter')

    # Write each DataFrame to a separate sheet
    for period, result in results_dict.items():
        result.to_excel(writer, sheet_name=period)

    # Save the Excel file
    writer.close()